In [1]:
# JIRA Bug Risk Analysis - Workspace Core Project
# Using Centralized Configuration System

print("🔧 JIRA Bug Risk Analysis - Workspace Core Project")
print("=" * 60)

# Import required libraries with error handling
try:
    import jinja2
    print(f"✅ Jinja2 version: {jinja2.__version__}")
    
    # Import centralized configuration system
    import sys
    sys.path.append('..')  # Add parent directory to path
    
    from notebook_helper import setup_notebook, fetch_and_display_jira_data, create_pie_chart
    from component_risk_table import component_risk_table
    from IPython.display import display, HTML
    
    print("✅ All imports successful!")
    
    # Setup notebook with centralized configuration
    setup_info = setup_notebook('AW', display_config=True)
    project_name = setup_info['project_info']['name']
    project_key = setup_info['project_key']
    
    # Fetch JIRA data using centralized configuration
    data = fetch_and_display_jira_data(project_key, save_to_csv=True)
    
    # Display title and create visualizations
    report_title_html = f"""
    <div style="background: #111; border-radius: 12px; padding: 18px 28px; margin-bottom: 24px; box-shadow: 0 2px 8px #222; text-align:center;">
        <h2 style="margin-top:0; color:#ffe066;">🔎 <b>Bug Risk Analysis: {project_name}</b></h2>
        <p style="font-size: 1.5em; color: #fff; margin: 0 auto; display: inline-block; text-align: center;">
            Analyzing bug distribution and risk scores to spotlight the most vulnerable components.<br>
            <b>Data window:</b> Last 6 months<br>
            <b>Use:</b> Guide testing, triage, and resource allocation.
        </p>
    </div>
    """
    display(HTML(report_title_html))

    # Create pie chart if data exists
    if not data.empty:
        create_pie_chart(data, project_name)
        
        # Generate component risk analysis
        component_risk_table(data, project_name)
    else:
        print("⚠️  No data available for analysis")
        
except ImportError as e:
    print(f"❌ Import Error: {e}")
    print("\n💡 Solutions:")
    print("1. Install missing packages: pip install python-dotenv jira flask")
    print("2. Restart the kernel after installation")
    
except Exception as e:
    print(f"❌ Unexpected Error: {e}")
    print("\n💡 Please check your environment setup")

# ✅ Analysis complete - using centralized configuration system!
# All visualization and analysis is handled by the helper functions above.

# 📋 Note: Old code has been replaced with centralized configuration system
# ✅ Migration to centralized configuration complete!
    data['Created'] = data['Created'].dt.tz_localize(None)

if 'Created' in data.columns and 'Components' in data.columns and not data['Created'].isnull().all():
    last_month = data['Created'].max().to_period('M')
    last_6_months = pd.period_range(end=last_month, periods=6, freq='M').astype(str)
    data['Created_Month'] = data['Created'].dt.to_period('M').astype(str)
    data_last6 = data[data['Created_Month'].isin(last_6_months)]

    component_counts = data_last6['Components'].value_counts()
    top_n = 10
    top_components = component_counts[:top_n]
    other_count = component_counts[top_n:].sum()
    if other_count > 0:
        top_components['Other'] = other_count

    # Create and save the plot to a buffer
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.pie(
        top_components,
        labels=top_components.index,
        autopct='%1.1f%%',
        startangle=140,
        counterclock=False,
        wedgeprops={'edgecolor': 'white'}
    )
    ax.set_title('Workspace Core (AW) Bugs Distribution by Component (Last 6 Months)', fontsize=18, fontweight='bold')
    plt.tight_layout()
    buf = io.BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight')
    plt.close(fig)
    buf.seek(0)
    img_base64 = base64.b64encode(buf.read()).decode('utf-8')
    buf.close()
    # Display the image centered
    display(HTML(f"""
    <div style="display: flex; flex-direction: column; align-items: center;">
        <img src="data:image/png;base64,{img_base64}" style="max-width:600px; height:auto; margin: 0 auto 32px auto;" />
    </div>
    """))

# Component risk analysis will be done in the component_risk_table function

def component_risk_table(data, project_name):
    # Check if data is empty or missing required columns
    if data.empty or 'Components' not in data.columns:
        display(HTML(f"<b>No data available for {project_name}. DataFrame is empty or missing 'Components' column.</b>"))
        return
    
    # Filter out rows with missing components
    data_filtered = data[data['Components'].notna()].copy()
    
    if data_filtered.empty:
        display(HTML(f"<b>No bug data with components available for {project_name}.</b>"))
        return
        
    # Use the filtered data for analysis (already filtered to last 6 months)
    data_last6 = data_filtered

    # Group by component and count bugs
    component_counts = data_last6['Components'].value_counts()
    component_risk_rank = component_counts.rank(ascending=False, method='min').astype(int)
    
    # Create component-level summary (not individual bug level)
    summary_df = pd.DataFrame({
        '🧩 Components': component_counts.index,
        '🐞 Bug Count': component_counts.values,
        'Risk Score': component_risk_rank.values
    })
    
    # Add explanation based on component risk with realistic assessments
    def get_component_explanation(component, count, risk_score, max_risk):
        if risk_score == 1:  # Highest risk (most bugs)
            if count >= 5:
                return f'The "{component}" component has the highest number of bugs ({count}), making it very high-risk.'
            elif count >= 3:
                return f'The "{component}" component has {count} bugs, making it high-risk.'
            else:
                return f'The "{component}" component has {count} bugs, making it medium-risk.'
        elif count == 1:  # Single bug components
            return f'The "{component}" component has only 1 bug, making it low-risk.'
        elif count == 2:  # Two bug components
            return f'The "{component}" component has 2 bugs, making it medium-risk.'
        elif count >= 3:  # Multiple bug components
            return f'The "{component}" component has {count} bugs, making it high-risk.'
        else:
            return f'The "{component}" component has {count} bug(s), making it low-risk.'
    
    max_risk = component_risk_rank.max()
    summary_df['Explanation'] = [
        get_component_explanation(row['🧩 Components'], row['🐞 Bug Count'], row['Risk Score'], max_risk) 
        for _, row in summary_df.iterrows()
    ]

    # Add color mapping based on bug count
    def get_risk_color_style(count):
        try:
            if count >= 5:  # Very high risk
                return 'background-color: #ff4d4d; color: white; font-weight: bold;'
            elif count >= 3:  # High risk
                return 'background-color: #ff8c66; color: white; font-weight: bold;'
            elif count == 2:  # Medium risk
                return 'background-color: #ffd966; color: black; font-weight: bold;'
            else:  # Low risk (1 bug)
                return 'background-color: #85e085; color: black; font-weight: bold;'
        except:
            return ''
    
    # Apply color styling based on bug count
    summary_df['_style'] = summary_df['🐞 Bug Count'].apply(get_risk_color_style)
    
    # Fancy summary
    total_bugs = len(data_last6)
    total_components = len(component_counts)
    top_component = component_counts.index[0]
    top_count = component_counts.iloc[0]
    
    # Determine risk level for top component
    if top_count >= 5:
        risk_level = "Very High Risk"
        risk_color = "#ff4d4d"
    elif top_count >= 3:
        risk_level = "High Risk"
        risk_color = "#ff8c66"
    elif top_count == 2:
        risk_level = "Medium Risk"
        risk_color = "#ffd966"
    else:
        risk_level = "Low Risk"
        risk_color = "#85e085"
    
    summary_html = f"""
<div style="background: #111; border-radius: 12px; padding: 18px 28px; margin-bottom: 24px; box-shadow: 0 2px 8px #222;">
    <h2 style="margin-top:0; color:#ffe066;">✨ <b>Component Risk Analysis for {project_name}</b> ✨</h2>
    <ul style="font-size: 1.1em; color: #fff;">
        <li><b>Total bugs (last 6 months):</b> {total_bugs}</li>
        <li><b>Total components affected:</b> {total_components}</li>
        <li><b>Top component:</b> <span style="color:{risk_color};">{top_component}</span> ({top_count} bugs - {risk_level})</li>
        <li><b>Risk levels:</b> 5+ bugs = Very High, 3-4 bugs = High, 2 bugs = Medium, 1 bug = Low</li>
    </ul>
</div>
"""

    # Create custom HTML table with color coding
    table_rows = []
    for i, row in summary_df.head(10).iterrows():
        component = row['🧩 Components']
        bug_count = row['🐞 Bug Count']
        risk_score = row['Risk Score']
        explanation = row['Explanation']
        style = row['_style']
        
        table_rows.append(f"""
        <tr>
            <td style="padding: 8px; border: 1px solid #ddd;">{i}</td>
            <td style="padding: 8px; border: 1px solid #ddd;">{component}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center;">{bug_count}</td>
            <td style="padding: 8px; border: 1px solid #ddd; text-align: center; {style}">{risk_score}</td>
            <td style="padding: 8px; border: 1px solid #ddd; max-width: 400px; word-wrap: break-word;">{explanation}</td>
        </tr>
        """)
    
    custom_table = f"""
    <table style="border-collapse: collapse; width: 100%; margin: 0 auto;">
        <thead>
            <tr style="background-color: #1a1368; color: white; font-size: 1.1em;">
                <th style="padding: 12px; border: 1px solid #ddd;"></th>
                <th style="padding: 12px; border: 1px solid #ddd;">🧩 Components</th>
                <th style="padding: 12px; border: 1px solid #ddd;">🐞 Bug Count</th>
                <th style="padding: 12px; border: 1px solid #ddd;">Risk Score</th>
                <th style="padding: 12px; border: 1px solid #ddd;">Explanation</th>
            </tr>
        </thead>
        <tbody>
            {''.join(table_rows)}
        </tbody>
    </table>
    """

    display(HTML(summary_html))
    display(HTML(f"""
    <div style="display: flex; flex-direction: column; align-items: center;">
        <h3 style="text-align:center; color:#2d4157;"><span style="color:#ffd166;">📊 <b>Component Risk Scoring Table</b></h3>
        <div style="min-width:450px; max-width:900px;">
            {custom_table}
        </div>
        <p style="color:#888; font-size:0.95em; margin-top:18px;">🔎 <i>Components with realistic risk assessment: Red = Very High (5+ bugs), Orange = High (3+ bugs), Yellow = Medium (2 bugs), Green = Low (1 bug).</i></p>
    </div>
    """))

component_risk_table(data, "Workspace Core (AW)")

🔧 JIRA Bug Risk Analysis - Workspace Core Project
✅ Jinja2 version: 3.1.6
✅ Environment variables loaded from .env file
✅ All imports successful!
🔧 JIRA Bug Risk Analysis - Notebook Setup
🔧 JIRA Configuration:
   URL: https://your-company.atlassian.net/
   Email: your-email@company.com
   Token: ********************...A=28FA7C69
   Projects: ['AND', 'iOS', 'WS', 'MSG', 'AWC']
   Environment: Production
   Months back: 6

📊 Setting up analysis for: Workspace Core (AW)
🔑 Project Key: AW
📅 Analysis Period: Last 6 months
🌍 Environment: Production

📡 Fetching JIRA data for project: AW
----------------------------------------
❌ Error fetching JIRA data for AW: JiraError HTTP 404 url: https://your-company.atlassian.net/rest/api/2/serverInfo
	text: Page not found
	
	response headers = {'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Date': 'Thu, 19 Jun 2025 17:17:57 GMT', 'Server': 'AtlassianEdge', 'X-Content-Type-Options': 'nosniff', 'X-Xss-Pro

⚠️  No data available for analysis
